In [223]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from random import randrange


In [224]:
#loading the data

data=pd.read_csv('data/imdb_labelled.txt' , delimiter='\t', names=['review', 'sentiment'])

In [225]:
data.head()


,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [226]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     748 non-null    object
 1   sentiment  748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [227]:
data['sentiment'].value_counts()

1    386
0    362
Name: sentiment, dtype: int64

In [242]:
def text_filtering(text):
    #stop_words = list(stopwords.words('english'))
    stop_words=[]
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    for word in text.split(" "):
        if word in stop_words:
            text=text.replace(word,'')
    text = text.rstrip() 
    regular_expression = '[^a-zA-z\s]'
    text = re.sub(regular_expression, '', text)
    return text
text_filtering("movie is not 73855 good")

'movie is not  good'

In [243]:
train_data = data[:int(0.5*len(data))]
test_and_dev_data = data[int(0.5*len(data)):]


dev_data=test_and_dev_data[:int(0.4 * len(test_and_dev_data))]
test_data=test_and_dev_data[int(0.4 * len(test_and_dev_data)):]

print(train_data.info())

print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     374 non-null    object
 1   sentiment  374 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.0+ KB
None



In [244]:
print(test_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 523 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     225 non-null    object
 1   sentiment  225 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ KB
None


In [245]:
print(dev_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 374 to 522
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     149 non-null    object
 1   sentiment  149 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None


In [246]:
train_data['sentiment'].value_counts()

0    221
1    153
Name: sentiment, dtype: int64

In [247]:
test_data['sentiment'].value_counts()

1    155
0     70
Name: sentiment, dtype: int64

In [248]:
dev_data['sentiment'].value_counts()

1    78
0    71
Name: sentiment, dtype: int64

In [249]:
X_train = train_data['review']
y_train = train_data['sentiment']
# print(X_train)
# print(y_train)

X_dev = dev_data['review']
y_dev = dev_data['sentiment']
# print(X_dev)
# print(y_dev)

X_test = test_data['review']
y_test = test_data['sentiment']
# print(X_test)
# print(y_test)

In [250]:

def cal_prior_probs(y_train):
    prior_probs = []
    count_0 = sum(y_train == 0)
    prior_probs.append(count_0 / y_train.size)  
    count_1 = sum(y_train == 1)
    prior_probs.append(count_1 / y_train.size)    
    return prior_probs
prior_probs=cal_prior_probs(y_train)
print("Negative review class' prior probability--",prior_probs[0] )
print("Positive review class' prior probability--",prior_probs[1] )

Negative review class' prior probability-- 0.5909090909090909
Positive review class' prior probability-- 0.4090909090909091


In [251]:
def build_word_dict(data):
    word_freq = {}
    all_words=[]
    omit=[]
    for sentence in data:
            words=[]
            sentence = text_filtering(sentence)
            words=sentence.split(' ')
            for word in set(words) :
                word = word.lower()
                if word not in word_freq.keys():
                    word_freq[word] = 1
                    all_words.append(word)
                else:
                    word_freq[word] += 1
#     for word in word_freq:
#         if word_freq[word] <5:
#             omit.append(word)

#     for word in omit:
#         word_freq.pop(word,None)
        
            

    return word_freq, all_words

    
       
# allwords_freq,allwords_train=build_word_dict(X_train)
# print("Total words--",len(allwords_train))


# train_pos_reviews=train_data.loc[train_data.sentiment==1].review
# pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
# print("Positive words--",len(pos_words))


# train_neg_reviews=train_data.loc[train_data.sentiment==0].review
# neg_words_freq,neg_words=build_word_dict(train_neg_reviews)
# print("Negative words--",len(neg_words))

In [252]:
def fit(train_data,X_train):
    allwords_freq,allwords_train=build_word_dict(X_train)
    train_pos_reviews=train_data.loc[train_data.sentiment==1].review
    pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
    train_neg_reviews=train_data.loc[train_data.sentiment==0].review
    neg_words_freq,neg_words=build_word_dict(train_neg_reviews)

    neg_words_freq.pop('', None)
    neg_condi_prob={}
    for word in neg_words_freq:
        #print(word,"--",neg_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",neg_words_freq[word]/len(train_neg_reviews))
        neg_condi_prob[word]=neg_words_freq[word]/len(train_neg_reviews) 
        
    pos_words_freq.pop('', None)
    pos_condi_prob={}
    for word in pos_words_freq:
        #print(word,"--",pos_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",pos_words_freq[word]/len(train_pos_reviews))
        pos_condi_prob[word]=pos_words_freq[word]/len(train_pos_reviews)
    
    
    return pos_condi_prob,neg_condi_prob
    
pos_condi_prob,neg_condi_prob=fit(train_data,X_train)   


In [253]:
def score(real, prediction):
    count = 0
    for i in range(len(real)):
        if real[i] == prediction[i]:
            count+=1
    accuracy=count/len(real)*100
    return accuracy

In [254]:
def pred(data,ans):
    y_preds=[]
    for sentence in data:
        sentence=text_filtering(sentence)
        pos_lik=1
        neg_lik=1
        for word in sentence.split(' '):
            word=word.lower()
            if word not in pos_condi_prob.keys():
                pos_condi_prob[word]=0
            if word not in neg_condi_prob.keys():
                neg_condi_prob[word]=0
            pos_lik*=pos_condi_prob[word]
            neg_lik*=neg_condi_prob[word]
        pos_posterior=pos_lik*prior_probs[1]
        neg_posterior=neg_lik*prior_probs[0]
        if pos_posterior>neg_posterior:
            y_preds.append(1)
        else:
            y_preds.append(0)
    
    return score(ans.tolist(),y_preds)
print("Accuracy on training data--",pred(X_train,y_train) )   
print("Accuracy on development data--",pred(X_dev,y_dev))
print("Accuracy on test data--",pred(X_test,y_test))

Accuracy on training data-- 92.24598930481284
Accuracy on development data-- 54.36241610738255
Accuracy on test data-- 32.88888888888889


In [255]:
def k_fold_acc(dataset):         # here k is 5
    
    dat_split=np.array_split(dataset,5)
    review_split=np.array_split(dataset['review'],5)
    senti_split=np.array_split(dataset['sentiment'],5)
    fold_acc=[]
    
    
    arrd1= np.concatenate((dat_split[1],dat_split[2],dat_split[3],dat_split[4]))
    arrx1=np.concatenate((review_split[1],review_split[2],review_split[3],review_split[4]))
    dfd1 = pd.DataFrame(arrd1, columns = ['review','sentiment'])
    dfx1= df = pd.DataFrame(arrx1, columns = ['review'])
    fit(dfd1,dfx1)
    acc=pred(review_split[0],senti_split[0])    
    fold_acc.append(acc)

    
    arrd2= np.concatenate((dat_split[0],dat_split[2],dat_split[3],dat_split[4]))
    arrx2=np.concatenate((review_split[0],review_split[2],review_split[3],review_split[4]))
    dfd2 = pd.DataFrame(arrd2, columns = ['review','sentiment'])
    dfx2= df = pd.DataFrame(arrx2, columns = ['review'])
    fit(dfd2,dfx2)
    acc=pred(review_split[1],senti_split[1])    
    fold_acc.append(acc)
    
    arrd3= np.concatenate((dat_split[0],dat_split[1],dat_split[3],dat_split[4]))
    arrx3=np.concatenate((review_split[0],review_split[1],review_split[3],review_split[4]))
    dfd3 = pd.DataFrame(arrd3, columns = ['review','sentiment'])
    dfx3= df = pd.DataFrame(arrx3, columns = ['review'])
    fit(dfd3,dfx3)
    acc=pred(review_split[2],senti_split[2])    
    fold_acc.append(acc)
    
    arrd4= np.concatenate((dat_split[0],dat_split[1],dat_split[2],dat_split[4]))
    arrx4=np.concatenate((review_split[0],review_split[1],review_split[2],review_split[4]))
    dfd4 = pd.DataFrame(arrd4, columns = ['review','sentiment'])
    dfx4= df = pd.DataFrame(arrx4, columns = ['review'])
    fit(dfd4,dfx4)
    acc=pred(review_split[3],senti_split[3])    
    fold_acc.append(acc)
    

    arrd5= np.concatenate((dat_split[0],dat_split[1],dat_split[2],dat_split[3]))
    arrx5=np.concatenate((review_split[0],review_split[1],review_split[2],review_split[3]))
    dfd5 = pd.DataFrame(arrd5, columns = ['review','sentiment'])
    dfx5= df = pd.DataFrame(arrx5, columns = ['review'])
    fit(dfd5,dfx5)
    acc=pred(review_split[4],senti_split[4])    
    fold_acc.append(acc)

    return fold_acc
        
        
   
    
    
    
print(k_fold_acc(dev_data) )  



[73.33333333333333, 50.0, 30.0, 40.0, 79.3103448275862]


In [ ]:
cd 